## 🔑 Part 2 연습문제

### 연습문제 정답

In [ ]:
# 연습문제 1 정답
imputer_median = SimpleImputer(strategy='median')
df_imputed_median = imputer_median.fit_transform(df_numeric)
df_imputed_median = pd.DataFrame(df_imputed_median, columns=df_numeric.columns)
# 중앙값으로 채워진 후 결측치가 없는지 확인
print("중앙값 처리 후 결측치 개수:", df_imputed_median['TotalCharges'].isnull().sum()) # 출력: 0


In [ ]:

# 연습문제 2 정답
dependents_series = df['Dependents']
le_dependents = LabelEncoder()
dependents_encoded = le_dependents.fit_transform(dependents_series)
print("원본 데이터:", dependents_series.unique()) # ['No' 'Yes']
print("인코딩된 데이터:", np.unique(dependents_encoded)) # [0 1]
print("인코딩 클래스 확인:", le_dependents.classes_) # ['No' 'Yes']


In [ ]:

# 연습문제 3 정답
payment_df = df[['PaymentMethod']]
ohe_payment = OneHotEncoder(sparse_output=False)
payment_onehot = ohe_payment.fit_transform(payment_df.fillna('Unknown'))
print("생성된 더미 변수 개수(shape):", payment_onehot.shape) # (7043, 4)
print("생성된 컬럼명:", ohe_payment.get_feature_names_out())
# ['PaymentMethod_Bank transfer (automatic)' 'PaymentMethod_Credit card (automatic)' 'PaymentMethod_Electronic check' 'PaymentMethod_Mailed check']


In [ ]:

# 연습문제 4 정답
df_subset = df_imputed[['tenure', 'TotalCharges']]
scaler_subset = StandardScaler()
df_scaled_subset = scaler_subset.fit_transform(df_subset)
print(df_scaled_subset[:5])


In [ ]:

# 연습문제 5 정답
print(pd.Series(y_train_resampled).value_counts(normalize=True))
# 0    0.5
# 1    0.5
# Name: proportion, dtype: float64


In [ ]:
# 연습문제 6 정답
from lib.preperate import ImbalancedDataAnalyzer
from lib.visualize import TSNEVisualizer
from sklearn.utils import resample

analyzer = ImbalancedDataAnalyzer(X_train, y_train)

X_adasyn, y_adasyn = analyzer.adasyn()
print(f"ADASYN 샘플링 후 데이터 크기: {X_adasyn.shape}")
print(f"ADASYN 클래스 분포:\n{pd.Series(y_adasyn).value_counts()}")

X_smote_tomek, y_smote_tomek = analyzer.smote_tomek()
print(f"\nSMOTE+Tomek 샘플링 후 데이터 크기: {X_smote_tomek.shape}")
print(f"SMOTE+Tomek 클래스 분포:\n{pd.Series(y_smote_tomek).value_counts()}")

X_smote_enn, y_smote_enn = analyzer.smote_enn()
print(f"\nSMOTE+ENN 샘플링 후 데이터 크기: {X_smote_enn.shape}")
print(f"SMOTE+ENN 클래스 분포:\n{pd.Series(y_smote_enn).value_counts()}")

df_adasyn = pd.DataFrame(X_adasyn, columns=X_train.columns)
df_adasyn['target'] = y_adasyn
X_adasyn_sample, y_adasyn_sample = resample(df_adasyn.drop('target', axis=1), 
                                           df_adasyn['target'], 
                                           n_samples=3000, 
                                           random_state=42)

df_smote_tomek = pd.DataFrame(X_smote_tomek, columns=X_train.columns)
df_smote_tomek['target'] = y_smote_tomek
X_smote_tomek_sample, y_smote_tomek_sample = resample(df_smote_tomek.drop('target', axis=1), 
                                                     df_smote_tomek['target'], 
                                                     n_samples=3000, 
                                                     random_state=42)

df_smote_enn = pd.DataFrame(X_smote_enn, columns=X_train.columns)
df_smote_enn['target'] = y_smote_enn
X_smote_enn_sample, y_smote_enn_sample = resample(df_smote_enn.drop('target', axis=1), 
                                                 df_smote_enn['target'], 
                                                 n_samples=3000, 
                                                 random_state=42)

print(f"\n3000개 샘플링 후:")
print(f"ADASYN 클래스 분포: {pd.Series(y_adasyn_sample).value_counts()}")
print(f"SMOTE+Tomek 클래스 분포: {pd.Series(y_smote_tomek_sample).value_counts()}")
print(f"SMOTE+ENN 클래스 분포: {pd.Series(y_smote_enn_sample).value_counts()}")

TSNEVisualizer.visualize(X_adasyn_sample, y_adasyn_sample, 'ADASYN t-SNE 시각화')
TSNEVisualizer.visualize(X_smote_tomek_sample, y_smote_tomek_sample, 'SMOTE+Tomek t-SNE 시각화')
TSNEVisualizer.visualize(X_smote_enn_sample, y_smote_enn_sample, 'SMOTE+ENN t-SNE 시각화')


In [ ]:

# 연습문제 7 정답
numeric_transformer_minmax = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])
preprocessor_minmax = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_minmax, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
model_pipeline_minmax = Pipeline(steps=[
    ('preprocessor', preprocessor_minmax),
    ('classifier', LogisticRegression(solver='liblinear'))
])
model_pipeline_minmax.fit(X_train, y_train)
accuracy_minmax = model_pipeline_minmax.score(X_test, y_test)
print(f"MinMaxScaler 파이프라인 모델의 정확도: {accuracy_minmax:.4f}") # 0.8204

In [ ]:
# 연습문제 8 정답

# 특성(X)과 타겟(y) 분리
X = df_heart.drop('DEATH_EVENT', axis=1)
y = df_heart['DEATH_EVENT']

# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# [문제 1] 수치형 특성과 범주형(이진) 특성의 컬럼명을 리스트로 정의하세요.
numeric_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

# [문제 2] ColumnTransformer를 사용하여 전처리기를 만드세요.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# [문제 3] Pipeline을 사용하여 전처리기와 RandomForestClassifier 모델을 연결하세요.
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# [문제 4] 생성한 파이프라인을 훈련 데이터로 학습시키세요.
model_pipeline.fit(X_train, y_train)

# [문제 5] 학습된 파이프라인으로 테스트 데이터의 예측을 수행하고 정확도를 계산하여 출력하세요.
y_pred = model_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"심부전 예측 모델 파이프라인의 정확도: {accuracy:.4f}")